# EDA: Pojazdy Elektryczne

Do zajęć wykorzystano zbiór danych postępny pod adresem: 

https://www.kaggle.com/datasets/urvishahir/electric-vehicle-specifications-dataset-2025/data?select=electric_vehicles_spec_2025.csv.csv



In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## Faza 1: Zrozumienie Danych

**Ładowanie i podstawowa inspekcja:**
- `pd.read_csv(...)`
- `df.head()`, `df.tail()`
- `df.shape`
- `df.info()`
- `df.describe()`

**Duplikaty:**
- `df.duplicated().sum()`
- `df[df.duplicated(keep=False)]`

**Brakujące wartości:**
- `df.isnull().sum()`
- `(df.isnull().sum() / len(df) * 100).round(2)`
- `df.isnull().sum().plot(kind='bar')`

**Histogramy dla zmiennych numerycznych:**
- `df['range_km'].hist(bins=30)`
- `df[['range_km', 'battery_capacity_kWh']].hist(figsize=(12, 8))`

**Analiza zmiennych kategorycznych:**
- `df['brand'].value_counts()`
- `df['brand'].value_counts().head(15).plot(kind='bar')`
- `df['battery_type'].value_counts(normalize=True) * 100`

## Faza 2: Przygotowanie Danych

**Kopia DataFrame:**
- `df_clean = df.copy()`

**Usuwanie kolumn z dużą liczbą braków:**
- `missing_pct = df_clean.isnull().sum() / len(df_clean)`
- `cols_to_drop = missing_pct[missing_pct > 0.20].index`
- `df_clean = df_clean.drop(columns=cols_to_drop)`

**Imputacja brakujących wartości:**
- `df_clean['kolumna'].fillna(df_clean['kolumna'].median())`

**Usuwanie duplikatów:**
- `df_clean = df_clean.drop_duplicates()`

**Konwersja typów:**
- `pd.to_numeric(df_clean['cargo_volume_l'], errors='coerce')`
- `df_clean['cargo_volume_l'] = pd.to_numeric(df_clean['cargo_volume_l'], errors='coerce')`

**Wykrywanie outlierów (metoda IQR):**
```python
def detect_outliers_iqr(data, column):
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    return data[(data[column] < lower) | (data[column] > upper)]
```

**Feature engineering:**
- `df_clean['range_efficiency'] = df_clean['range_km'] / df_clean['battery_capacity_kWh']`
- `df_clean.nlargest(5, 'range_efficiency')`
- `df_clean.nsmallest(5, 'range_efficiency')`

## Faza 3: Analiza Danych

**Analiza korelacji:**
- `df.corr()`
- `sns.heatmap(df.corr(), annot=True, cmap='coolwarm', center=0)`

**Korelacja między dwiema zmiennymi:**
- `df_clean['battery_capacity_kWh'].corr(df_clean['range_km'])`
- `plt.scatter(df_clean['battery_capacity_kWh'], df_clean['range_km'])`

**Analiza grupowa:**
- `df_clean.groupby('battery_type')['range_km'].mean()`
- `df_clean.groupby('battery_type')['range_km'].agg(['mean', 'median', 'std', 'min', 'max'])`
- `df_clean.groupby('segment').agg({'range_km': 'mean', 'battery_capacity_kWh': 'mean'})`

**Box plot dla grup:**
- `df_clean.boxplot(column='range_km', by='battery_type')`
- `sns.boxplot(data=df_clean, x='battery_type', y='range_km')`

**Wykresy grupowe:**
- `df_clean.groupby('brand')['range_efficiency'].mean().sort_values().plot(kind='barh')`
- `df_clean.groupby('segment')['range_km'].mean().plot(kind='bar')`

**Top N wartości:**
- `df_clean['brand'].value_counts().head(10)`
- `df_clean[df_clean['brand'].isin(top_brands)]`